In [ ]:
# default_exp hyperoptimization

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Hyperparameters optimization

>Description

In [ ]:
#export
# imports
import torch
import time
from functools import partial
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope
import json
from datetime import datetime

In [ ]:
#export
def fit_predict(mc, verbose, trials=None):

    now = datetime.now().strftime("%d-%m-%y_%H-%M-%S")
    print(now)

    start_time = time.time()
    print('='*26)
    print(pd.Series(mc))
    print('='*26+'\n')

    train_dataset = FaceClassificationDataset(mode='train')
    val_dataset = FaceClassificationDataset(mode='val')

    train_loader = DataLoader(train_dataset, 
                          shuffle=True, 
                          batch_size=mc['batch_size'], 
                          num_workers=num_workers, 
                          pin_memory=True,
                          drop_last=True)

    val_loader = DataLoader(val_dataset, 
                            shuffle=False, 
                            batch_size=mc['batch_size'], 
                            num_workers=num_workers, 
                            pin_memory=True,
                            drop_last=True)

    model = FaceClassificationCNN(n_ch_input=mc['n_ch_input'],
                                  n_classes=mc['n_classes'],
                                  lr=mc['lr'],
                                  lr_decay=float(mc['lr_decay']),
                                  n_lr_decay_steps=int(mc['n_lr_decay_steps']),
                                  n_epochs=mc['n_epochs'],
                                  eval_steps=mc['eval_steps'])

    model.fit(train_loader=train_loader, val_loader=val_loader)

    s = 'hw2p2-s1' + '_' + now
    filename = f'../results/{s}.pth' 

    best_model_config = {'loss': model.val_loss,
                         'mc': mc,
                         'run_time': time.time()-start_time,
                         'trajectories': model.trajectories,
                         'status': STATUS_OK}
        
    best_model_config = json.dumps(best_model_config)
    with open(f'best_model_config_{now}.json', 'w') as bfm: 
        bfm.write(best_model_config)    
    torch.save(model.model.state_dict(), filename)

    if trials is not None:
        results = {'loss': model.val_loss,
                   'mc': mc,
                   'run_time': time.time()-start_time,
                   'trajectories': model.trajectories,
                   'status': STATUS_OK}
        return results
    else:
        return model

In [ ]:
import numpy as np
import pandas as pd

space = {'n_ch_input': hp.choice(label='n_ch_input', options=[3]),
         'n_classes': hp.choice(label='n_classes', options=[4000]),
         'batch_size': scope.int(hp.choice(label='batch_size', options=[1024])),
         'lr': hp.loguniform(label='lr', low=np.log(5e-4), high=np.log(0.01)),
         'lr_decay': hp.choice(label='lr_decay', options=[1]),
         'n_lr_decay_steps': hp.choice(label='n_lr_decay_steps', options=[2]),
         'n_epochs': hp.choice(label='n_epochs', options=[16]),
         'eval_steps': scope.int(hp.choice(label='eval_steps', options=[4])),}

print(26*'=')
print(np.expand_dims(pd.Series(space).index.to_numpy(), axis = 1))
print(26*'=')

[['n_ch_input']
 ['n_classes']
 ['batch_size']
 ['lr']
 ['lr_decay']
 ['n_lr_decay_steps']
 ['n_epochs']
 ['eval_steps']]


In [ ]:
from axa_hw2p2.datasets import FaceClassificationDataset
from axa_hw2p2.models import _FaceClassificationCNN, FaceClassificationCNN
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision
from PIL import Image
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

In [ ]:
num_workers = 8 if torch.cuda.is_available() else 0
trials = Trials()
fmin_objective = partial(fit_predict, trials=trials, verbose=True)
fmin(fmin_objective, space=space, 
     algo=tpe.suggest, max_evals=20, trials=trials)

05-03-21_16-03-33
batch_size          1024.000000
eval_steps             4.000000
lr                     0.000619
lr_decay               1.000000
n_ch_input             3.000000
n_classes           4000.000000
n_epochs              16.000000
n_lr_decay_steps       2.000000
dtype: float64

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]job exception: name 'torch' is not defined

  0%|          | 0/20 [00:22<?, ?trial/s, best loss=?]


NameError: name 'torch' is not defined